# Calculating Fannie Mae Net Loss

In this notebook, we will calculate the net loss for each loan in the Fannie Mae data set. In order to do this, we are going to have to combine multiple different data sources. Specifically, we are going to have to combine data from the acquisitions data set provided by Fannie Mae and the performance data set.

We will import only a few things here because we are primarily working with data. We won't be building any models. We will also define our `summarize_dataframe` function.

While we could just use the full data set, we will find it useful to do it quarter by quarter. This will speed up the joining process and allow us to work on machines that have more limited memory. This is also good practice when working with large data sets.

In [1]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import datetime
import math
import os

FILES_LOCATION = '../Shared Data (Read Only)/Fannie Mae Preprocessing Data/'

In [2]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all', datetime_is_numeric=True).transpose()], axis=1).fillna(""))

## Read in the Data

In this starter code, we will just be working with the 2008 Q2 data. This data will be in the class shared folder. In older versions of the Fannie Mae data this consisted of two separate files, one consisting of data that Fannie Mae has at mortgage acquisition and one about ongoing performance data related to the mortgage. The data has been migrated to a single file that appends the acquisition data to each row of performance data. This has made the data more unwieldy to work with. We will read in this combined file called "2008Q2.csv" and we will split it out into separate performance and acquisition data frames.

Before we read in the data, we are going to go ahead and set the names for the columns of the data. We will also set the data type of each column. This allows pandas to be much more efficient with loading the data set, and it will cause pandas to use much less memory when storing the data. When working with data this large, these kinds of optimizations can make the difference between being able to read in the data and work with versus not even being able to read it in. However, we don't expect you to fully understand the specific data types, but the basic idea is to let pandas know when a data type is a number, character string, or categorical data. When panda knows the data type, it can be much more efficient with storing the data.

To determine the data types, we had to just go through the data dictionary (available [here](https://capitalmarkets.fanniemae.com/media/6931/display)) and look at the "Type" of data as well as its "Max Length". This is a very tedious process, but it is often an essential part of working with large data sets.

We also go ahead and tell pandas which columns are date columns. This tells pandas to try to parse the columns as some kind of timestamp which will be helpful once we start working with the data.

Finally, from looking at the column names, you can see that there are a lot of variables. Only some of the variables apply to our dataset. We are working the "Single Family (SF) Loan Performance" data, and you can see in the data dictionary that some data fields are listed as "NA" for this data set. So, those fields will not be in our data set. However, Fanne Mae still distributes the data with empty columns, so we still need to tell pandas to expect those empty columns. Therefore, we include all 108 fields from the data dictionary, even though our data will only contain 71 fields.

In [3]:
col_names = ["REFERENCE_POOL_ID",
             "LOAN_IDENTIFIER", 
             "MONTHLY_REPORTING_PERIOD",
             "CHANNEL", 
             "SELLER_NAME", 
             "SERVICER_NAME",
             "MASTER_SERVICER",
             "ORIGINAL_INTEREST_RATE", 
             "CURRENT_INTEREST_RATE", 
             "ORIGINAL_UPB",
             "UPB_AT_ISSUANCE",
             "CURRENT_ACTUAL_UPB", 
             "ORIGINAL_LOAN_TERM",
             "ORIGINATION_DATE",
             "FIRST_PAYMENT_DATE", 
             "LOAN_AGE", 
             "REMAINING_MONTHS_TO_LEGAL_MATURITY", 
             "REMAINING_MONTHS_TO_MATURITY", 
             "MATURITY_DATE", 
             "ORIGINAL_LTV", 
             "ORIGINAL_COMBINED_LTV", 
             "NUMBER_OF_BORROWERS", 
             "DTI", 
             "BORROWER_CREDIT_SCORE_AT_ORIGINATION", 
             "COBORROWER_CREDIT_SCORE_AT_ORIGINATION", 
             "FIRST_TIME_HOME_BUYER_INDICATOR", 
             "LOAN_PURPOSE", 
             "PROPERTY_TYPE",
             "NUMBER_OF_UNITS", 
             "OCCUPANCY_STATUS", 
             "PROPERTY_STATE", 
             "MSA", 
             "ZIP_CODE_SHORT", 
             "MORTGAGE_INSURANCE_PERCENTAGE", 
             "AMORTIZATION_TYPE",
             "PREPAYMENT_PENALTY_INDICATOR",
             "INTEREST_ONLY_LOAN_INDICATOR",
             "INTEREST_ONLY_FIRST_PRINCIPAL_AND_INTEREST_PAYMENT_DATE",
             "MONTHS_TO_AMORTIZATION",
             "CURRENT_LOAN_DELINQUENCY_STATUS", 
             "LOAN_PAYMENT_HISTORY",
             "MODIFICATION_FLAG", 
             "MORTGAGE_INSURANCE_CANCELLATION_INDICATOR",
             "ZERO_BALANCE_CODE",
             "ZERO_BALANCE_EFFECTIVE_DATE", 
             "UPB_AT_TIME_OF_REMOVAL",
             "REPURCHASE_DATE",
             "SCHEDULED_PRINCIPAL_CURRENT",
             "TOTAL_PRINCIPAL_CURRENT",
             "UNSCHEDULED_PRINCIPAL_CURRENT",          
             "LAST_PAID_INSTALLMENT_DATE", 
             "FORECLOSURE_DATE",
             "DISPOSITION_DATE", 
             "FORECLOSURE_COSTS", 
             "PROPERTY_PRESERVATION_AND_REPAIR_COSTS", 
             "ASSET_RECOVERY_COSTS", 
             "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS", 
             "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", 
             "NET_SALE_PROCEEDS",
             "CREDIT_ENHANCEMENT_PROCEEDS", 
             "REPURCHASE_MAKE_WHOLE_PROCEEDS", 
             "OTHER_FORECLOSURE_PROCEEDS", 
             "NON_INTEREST_BEARING_UPB", 
             "PRINCIPLE_FORGIVENESS_AMOUNT", 
             "ORIGINAL_LIST_START_DATE",
             "ORIGINAL_LIST_PRICE",
             "CURRENT_LIST_START_DATE",
             "CURRENT_LIST_PRICE",
             "BORROWER_CREDIT_SCORE_AT_ISSUANCE",
             "COBORROWER_CREDIT_SCORE_AT_ISSUANCE",
             "BORROWER_CREDIT_SCORE_CURRENT",
             "COBORROWER_CREDIT_SCORE_CURRENT",
             "MORTGAGE_INSURANCE_TYPE", 
             "SERVICING_ACTIVITY_INDICATOR",
             "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
             "CUMULATIVE_MODIFICATION_LOSS_AMOUNT",
             "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
             "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS",
             "HOMEREADY_PROGRAM_INDICATOR",
             "FORECLOSURE_PRINCIPAL_WRITE_OFF",
             "RELOCATION_MORTGAGE_INDICATOR",
             "ZERO_BALANCE_CODE_CHANGE_DATE",
             "LOAN_HOLDBACK_INDICATOR",
             "LOAN_HOLDBACK_EFFECTIVE_DATE",
             "DELINQUENT_ACCRUED_INTEREST",
             "PROPERTY_VALUATION_METHOD",
             "HIGH_BALANCE_LOAN_INDICATOR",
             "ARM_INITAL_FIXED_RATE_PERIOD_LT_5_YR",
             "ARM_PRODUCT_TYPE",
             "INITIAL_FIXED_RATE_PERIOD",
             "INTEREST_RATE_ADJUSTMENT_FREQUENCY",
             "NEXT_INTEREST_RATE_ADJUSTMENT_DATE",
             "NEXT_PAYMENT_CHANGE_DATE",
             "INDEX",
             "ARM_CAP_STRUCTURE",
             "INITIAL_INTEREST_RATE_CAP_UP_PERCENT",
             "PERIODIC_INTEREST_RATE_CAP_UP_PERCENT",
             "LIFETIME_INTEREST_RATE_CAP_UP_PERCENT",
             "MORTGAGE_MARGIN",
             "ARM_BALLOON_INDICATOR",
             "ARM_PLAN_NUMBER",
             "BORROWER_ASSISTANCE_PLAN",
             "HLTV_REFINANCE_OPTION_INDICATOR",
             "DEAL_NAME",
             "REPURCHASE_MAKE_WHOLE_PROCEEDS_FLAG",
             "ALTERNATIVE_DELINQUENCY_RESOLUTION",
             "ALTERNATIVE_DELINQUENCY_RESOLUTION_COUNT",
             "TOTAL_DEFERRAL_AMOUNT"
             ]

col_classes = {"REFERENCE_POOL_ID": np.character,
               "LOAN_IDENTIFIER": np.character, 
               "MONTHLY_REPORTING_PERIOD": np.character, 
               "ORIGINATION_CHANNEL": 'category', 
               "SELLER_NAME": np.character, 
               "SERVICER_NAME": np.character, 
               "MASTER_SERVICER": np.character,
               "ORIGINAL_INTEREST_RATE": np.float32, 
               "CURRENT_INTEREST_RATE": np.float32, 
               "ORIGINAL_UPB": np.float64,
               "UPB_AT_ISSUANCE": np.float64,
               "CURRENT_ACTUAL_UPB": np.float64, 
               "ORIGINAL_LOAN_TERM": "Int16", 
               "ORIGINATION_DATE": np.character,
               "FIRST_PAYMENT_DATE": np.character, 
               "LOAN_AGE": "Int16", 
               "REMAINING_MONTHS_TO_LEGAL_MATURITY": "Int16", 
               "REMAINING_MONTHS_TO_MATURITY": "Int16", 
               "MATURITY_DATE": np.character, 
               "ORIGINAL_LTV": np.float32, 
               "ORIGINAL_COMBINED_LTV": np.float32, 
               "NUMBER_OF_BORROWERS": 'category', 
               "DTI": np.float32, 
               "BORROWER_CREDIT_SCORE_AT_ORIGINATION": "UInt16", 
               "COBORROWER_CREDIT_SCORE_AT_ORIGINATION": 'UInt16', 
               "FIRST_TIME_HOME_BUYER_INDICATOR": 'category', 
               "LOAN_PURPOSE": 'category', 
               "PROPERTY_TYPE": 'category',
               "NUMBER_OF_UNITS": "UInt16", 
               "OCCUPANCY_STATUS": 'category', 
               "PROPERTY_STATE": 'category', 
               "MSA": 'category', 
               "ZIP_CODE_SHORT": 'category', 
               "MORTGAGE_INSURANCE_PERCENTAGE": np.float32, 
               "AMORTIZATION_TYPE": np.character,
               "PREPAYMENT_PENALTY_INDICATOR": 'category',
               "INTEREST_ONLY_LOAN_INDICATOR": 'category',
               "INTEREST_ONLY_FIRST_PRINCIPAL_AND_INTEREST_PAYMENT_DATE": np.character,
               "MONTHS_TO_AMORTIZATION": "UInt16",
               "CURRENT_LOAN_DELINQUENCY_STATUS": 'category', 
               "LOAN_PAYMENT_HISTORY": 'category',
               "MODIFICATION_FLAG": 'category', 
               "MORTGAGE_INSURANCE_CANCELLATION_INDICATOR": 'category',
               "ZERO_BALANCE_CODE": 'category',
               "ZERO_BALANCE_EFFECTIVE_DATE": np.character,
               "UPB_AT_TIME_OF_REMOVAL": np.float32, 
               "REPURCHASE_DATE": np.character,
               "SCHEDULED_PRINCIPAL_CURRENT": np.float32,
               "TOTAL_PRINCIPAL_CURRENT": np.float32,
               "UNSCHEDULED_PRINCIPAL_CURRENT": np.float32,
               "LAST_PAID_INSTALLMENT_DATE": np.character, 
               "FORECLOSURE_DATE": np.character,
               "DISPOSITION_DATE": np.character, 
               "FORECLOSURE_COSTS": np.float32, 
               "PROPERTY_PRESERVATION_AND_REPAIR_COSTS": np.float32, 
               "ASSET_RECOVERY_COSTS": np.float32, 
               "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS": np.float32, 
               "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY": np.float32, 
               "NET_SALE_PROCEEDS": np.float32,
               "CREDIT_ENHANCEMENT_PROCEEDS": np.float32, 
               "REPURCHASE_MAKE_WHOLE_PROCEEDS": np.float32, 
               "OTHER_FORECLOSURE_PROCEEDS": np.float32, 
               "NON_INTEREST_BEARING_UPB": np.float32, 
               "PRINCIPLE_FORGIVENESS_AMOUNT": np.float32, 
               "ORIGINAL_LIST_START_DATE": np.character,
               "ORIGINAL_LIST_PRICE": np.float32,
               "CURRENT_LIST_START_DATE": np.character,
               "CURRENT_LIST_PRICE": np.float32,
               "BORROWER_CREDIT_SCORE_AT_ISSUANCE": "Int16",
               "COBORROWER_CREDIT_SCORE_AT_ISSUANCE": "Int16",
               "BORROWER_CREDIT_SCORE_CURRENT": "Int16",
               "COBORROWER_CREDIT_SCORE_CURRENT": "Int16",
               "MORTGAGE_INSURANCE_TYPE": 'category', 
               "SERVICING_ACTIVITY_INDICATOR": 'category',
               "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT": np.float32,
               "CUMULATIVE_MODIFICATION_LOSS_AMOUNT": np.float32,
               "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS": np.float32,
               "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS": np.float32,
               "HOMEREADY_PROGRAM_INDICATOR": 'category',
               "FORECLOSURE_PRINCIPAL_WRITE_OFF": np.float32,
               "RELOCATION_MORTGAGE_INDICATOR": 'category',
               "ZERO_BALANCE_CODE_CHANGE_DATE": np.character,
               "LOAN_HOLDBACK_INDICATOR": 'category',
               "LOAN_HOLDBACK_EFFECTIVE_DATE": np.character,
               "DELINQUENT_ACCRUED_INTEREST": np.float32,
               "PROPERTY_VALUATION_METHOD": 'category',
               "HIGH_BALANCE_LOAN_INDICATOR": 'category',
               "ARM_INITAL_FIXED_RATE_PERIOD_LT_5_YR": np.character,
               "ARM_PRODUCT_TYPE": np.character,
               "INITIAL_FIXED_RATE_PERIOD": np.float32,
               "INTEREST_RATE_ADJUSTMENT_FREQUENCY": np.float32,
               "NEXT_INTEREST_RATE_ADJUSTMENT_DATE": np.character,
               "NEXT_PAYMENT_CHANGE_DATE": np.character,
               "INDEX": np.character,
               "ARM_CAP_STRUCTURE": np.character,
               "INITIAL_INTEREST_RATE_CAP_UP_PERCENT": np.float32,
               "PERIODIC_INTEREST_RATE_CAP_UP_PERCENT": np.float32,
               "LIFETIME_INTEREST_RATE_CAP_UP_PERCENT": np.float32,
               "MORTGAGE_MARGIN": np.float32,
               "ARM_BALLOON_INDICATOR": 'category',
               "ARM_PLAN_NUMBER": np.float32,
               "BORROWER_ASSISTANCE_PLAN": 'category',
               "HLTV_REFINANCE_OPTION_INDICATOR": 'category',
               "DEAL_NAME": np.character,
               "REPURCHASE_MAKE_WHOLE_PROCEEDS_FLAG": 'category',
               "ALTERNATIVE_DELINQUENCY_RESOLUTION": 'category',
               "ALTERNATIVE_DELINQUENCY_RESOLUTION_COUNT": 'Int16',
               "TOTAL_DEFERRAL_AMOUNT": np.float32
               }

date_columns = ["MONTHLY_REPORTING_PERIOD", 
                "ORIGINATION_DATE",
                "FIRST_PAYMENT_DATE",
                "MATURITY_DATE", 
                "INTEREST_ONLY_FIRST_PRINCIPAL_AND_INTEREST_PAYMENT_DATE",
                "ZERO_BALANCE_EFFECTIVE_DATE", 
                "REPURCHASE_DATE",
                "LAST_PAID_INSTALLMENT_DATE", 
                "FORECLOSURE_DATE", 
                "DISPOSITION_DATE",
                "ORIGINAL_LIST_START_DATE",
                "CURRENT_LIST_START_DATE",
                "ZERO_BALANCE_CODE_CHANGE_DATE",
                "LOAN_HOLDBACK_EFFECTIVE_DATE",
                "NEXT_INTEREST_RATE_ADJUSTMENT_DATE",
                "NEXT_PAYMENT_CHANGE_DATE"]

Now it is time to read in the data. Note that while this file is similar is a "csv" file, it is not actually comma separated. The actual delimiter is `|`, and we have to tell pandas that. Why would the file be delimited with a `|` instead of a `,`?

Reading in the data may take some time. That is expected.

In [4]:
%%time
df_full = pd.read_csv(FILES_LOCATION + "2008Q2.csv",
                 header=None, names=col_names,
                 dtype=col_classes,
                 parse_dates=date_columns,
                 delimiter='|'
                )

CPU times: user 4min 29s, sys: 23.9 s, total: 4min 53s
Wall time: 4min 52s


In [5]:
summarize_dataframe(df_full)

Data Type  \
REFERENCE_POOL_ID                                           object   
LOAN_IDENTIFIER                                             object   
MONTHLY_REPORTING_PERIOD                            datetime64[ns]   
CHANNEL                                                     object   
SELLER_NAME                                                 object   
SERVICER_NAME                                               object   
MASTER_SERVICER                                             object   
ORIGINAL_INTEREST_RATE                                     float32   
CURRENT_INTEREST_RATE                                      float32   
ORIGINAL_UPB                                               float64   
UPB_AT_ISSUANCE                                            float64   
CURRENT_ACTUAL_UPB                                         float64   
ORIGINAL_LOAN_TERM                                           Int16   
ORIGINATION_DATE                                    datetime64[ns]   
FIRST_PAYMENT_DATE                                  datetime64[ns]   
LOAN_AGE                                                     Int16   
REMAINING_MONTHS_TO_LEGAL_MATURITY                           Int16   
REMAINING_MONTHS_TO_MATURITY                                 Int16   
MATURITY_DATE                                       datetime64[ns]   
ORIGINAL_LTV                                               float32   
ORIGINAL_COMBINED_LTV                                      float32   
NUMBER_OF_BORROWERS                                       category   
DTI                                                        float32   
BORROWER_CREDIT_SCORE_AT_ORIGINATION                        UInt16   
COBORROWER_CREDIT_SCORE_AT_ORIGINATION                      UInt16   
FIRST_TIME_HOME_BUYER_INDICATOR                           category   
LOAN_PURPOSE                                              category   
PROPERTY_TYPE                                             category   
NUMBER_OF_UNITS                                             UInt16   
OCCUPANCY_STATUS                                          category   
PROPERTY_STATE                                            category   
MSA                                                       category   
ZIP_CODE_SHORT                                            category   
MORTGAGE_INSURANCE_PERCENTAGE                              float32   
AMORTIZATION_TYPE                                           object   
PREPAYMENT_PENALTY_INDICATOR                              category   
INTEREST_ONLY_LOAN_INDICATOR                              category   
INTEREST_ONLY_FIRST_PRINCIPAL_AND_INTEREST_PAYM...  datetime64[ns]   
MONTHS_TO_AMORTIZATION                                      UInt16   
CURRENT_LOAN_DELINQUENCY_STATUS                           category   
LOAN_PAYMENT_HISTORY                                      category   
MODIFICATION_FLAG                                         category   
MORTGAGE_INSURANCE_CANCELLATION_INDICATOR                 category   
ZERO_BALANCE_CODE                                         category   
ZERO_BALANCE_EFFECTIVE_DATE                         datetime64[ns]   
UPB_AT_TIME_OF_REMOVAL                                     float32   
REPURCHASE_DATE                                     datetime64[ns]   
SCHEDULED_PRINCIPAL_CURRENT                                float32   
TOTAL_PRINCIPAL_CURRENT                                    float32   
UNSCHEDULED_PRINCIPAL_CURRENT                              float32   
LAST_PAID_INSTALLMENT_DATE                          datetime64[ns]   
FORECLOSURE_DATE                                    datetime64[ns]   
DISPOSITION_DATE                                    datetime64[ns]   
FORECLOSURE_COSTS                                          float32   
PROPERTY_PRESERVATION_AND_REPAIR_COSTS                     float32   
ASSET_RECOVERY_COSTS                                       float32   
MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS                 float32 

We will create lists of data fields that are related to loan origination and acquisition and data fields related to performance data. This will allow us to separate the data into two separate dataframes, one for acquisition and one for performance. We did this by going through and manually determining where each data type belongs using the data dictionary. This is a very tedious process and it requires some understanding of the data fields in order to accomplish successfully.

In [6]:
acquisition_columns = ["LOAN_IDENTIFIER", 
                       "MONTHLY_REPORTING_PERIOD",
                       "CHANNEL", 
                       "SELLER_NAME", 
                       "ORIGINAL_INTEREST_RATE", 
                       "ORIGINAL_UPB",
                       "ORIGINAL_LOAN_TERM",
                       "ORIGINATION_DATE",
                       "FIRST_PAYMENT_DATE", 
                       "ORIGINAL_LTV", 
                       "ORIGINAL_COMBINED_LTV", 
                       "NUMBER_OF_BORROWERS", 
                       "DTI", 
                       "BORROWER_CREDIT_SCORE_AT_ORIGINATION", 
                       "COBORROWER_CREDIT_SCORE_AT_ORIGINATION", 
                       "FIRST_TIME_HOME_BUYER_INDICATOR", 
                       "LOAN_PURPOSE", 
                       "PROPERTY_TYPE",
                       "NUMBER_OF_UNITS", 
                       "OCCUPANCY_STATUS", 
                       "PROPERTY_STATE", 
                       "MSA", 
                       "ZIP_CODE_SHORT", 
                       "MORTGAGE_INSURANCE_PERCENTAGE", 
                       "AMORTIZATION_TYPE",
                       "MORTGAGE_INSURANCE_TYPE", 
                       "RELOCATION_MORTGAGE_INDICATOR"
                       ]

We do the same for the performance colums. Note that we don't include all performance columns because some will not be useful for computing the metrics that we are interested in.

In [7]:
perf_cols = ["LOAN_IDENTIFIER", 
             "MONTHLY_REPORTING_PERIOD",
             "SERVICER_NAME",
             "CURRENT_INTEREST_RATE", 
             "CURRENT_ACTUAL_UPB", 
             "LOAN_AGE", 
             "REMAINING_MONTHS_TO_LEGAL_MATURITY", 
             "REMAINING_MONTHS_TO_MATURITY", 
             "MATURITY_DATE", 
             "CURRENT_LOAN_DELINQUENCY_STATUS", 
             "MODIFICATION_FLAG", 
             "ZERO_BALANCE_CODE",
             "ZERO_BALANCE_EFFECTIVE_DATE", 
             "UPB_AT_TIME_OF_REMOVAL",
             "REPURCHASE_DATE",
             "LAST_PAID_INSTALLMENT_DATE", 
             "FORECLOSURE_DATE",
             "DISPOSITION_DATE", 
             "FORECLOSURE_COSTS", 
             "PROPERTY_PRESERVATION_AND_REPAIR_COSTS", 
             "ASSET_RECOVERY_COSTS", 
             "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS", 
             "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", 
             "NET_SALE_PROCEEDS",
             "CREDIT_ENHANCEMENT_PROCEEDS", 
             "REPURCHASE_MAKE_WHOLE_PROCEEDS", 
             "OTHER_FORECLOSURE_PROCEEDS", 
             "NON_INTEREST_BEARING_UPB", 
             "PRINCIPLE_FORGIVENESS_AMOUNT", 
             "FORECLOSURE_PRINCIPAL_WRITE_OFF",
             "REPURCHASE_MAKE_WHOLE_PROCEEDS_FLAG"
             ]

Now we can pull out the acquisition relevant fields.

In [8]:
df_acqu = df_full[acquisition_columns]

In [9]:
df_acqu.head()

LOAN_IDENTIFIER MONTHLY_REPORTING_PERIOD CHANNEL            SELLER_NAME  \
0    100000433226               2008-04-20       R  Bank Of America, N.A.   
1    100000433226               2008-05-20       R  Bank Of America, N.A.   
2    100000433226               2008-06-20       R  Bank Of America, N.A.   
3    100000433226               2008-07-20       R  Bank Of America, N.A.   
4    100000433226               2008-08-20       R  Bank Of America, N.A.   

   ORIGINAL_INTEREST_RATE  ORIGINAL_UPB  ORIGINAL_LOAN_TERM ORIGINATION_DATE  \
0                     6.5      153000.0                 360       2008-03-20   
1                     6.5      153000.0                 360       2008-03-20   
2                     6.5      153000.0                 360       2008-03-20   
3                     6.5      153000.0                 360       2008-03-20   
4                     6.5      153000.0                 360       2008-03-20   

  FIRST_PAYMENT_DATE  ORIGINAL_LTV  ORIGINAL_COMBINED_LTV NUMBER_OF_BORROWERS  \
0         2008-05-20          83.0                   83.0                   1   
1         2008-05-20          83.0                   83.0                   1   
2         2008-05-20          83.0                   83.0                   1   
3         2008-05-20          83.0                   83.0                   1   
4         2008-05-20          83.0                   83.0                   1   

    DTI  BORROWER_CREDIT_SCORE_AT_ORIGINATION  \
0  35.0                                   749   
1  35.0                                   749   
2  35.0                                   749   
3  35.0                                   749   
4  35.0                                   749   

   COBORROWER_CREDIT_SCORE_AT_ORIGINATION FIRST_TIME_HOME_BUYER_INDICATOR  \
0                                    <NA>                               N   
1                                    <NA>                               N   
2                                    <NA>                               N   
3                                    <NA>                               N   
4                                    <NA>                               N   

  LOAN_PURPOSE PROPERTY_TYPE  NUMBER_OF_UNITS OCCUPANCY_STATUS PROPERTY_STATE  \
0            C            SF                1                P             TN   
1            C            SF                1                P             TN   
2            C            SF                1                P             TN   
3            C            SF                1                P             TN   
4            C            SF                1                P             TN   

     MSA ZIP_CODE_SHORT  MORTGAGE_INSURANCE_PERCENTAGE AMORTIZATION_TYPE  \
0  32820            381                           12.0               FRM   
1  32820            381                           12.0               FRM   
2  32820            381                           12.0               FRM   
3  32820            381                           12.0               FRM   
4  32820            381                           12.0               FRM   

  MORTGAGE_INSURANCE_TYPE RELOCATION_MORTGAGE_INDICATOR  
0                       2                             N  
1                       2                             N  
2                       2                             N  
3                       2                             N  
4                       2                             N

We will now sort the data by both the loan identifier and the monthly reporting periods. We will then take the first row of each of the loans. This will leave us with a single row per loan.

In [10]:
df_acqu = df_acqu.sort_values(by=["LOAN_IDENTIFIER", "MONTHLY_REPORTING_PERIOD"]).groupby('LOAN_IDENTIFIER').head(1)
df_acqu = df_acqu.set_index("LOAN_IDENTIFIER").sort_index()
df_acqu.drop(["MONTHLY_REPORTING_PERIOD"], axis=1, inplace=True)

In [11]:
df_acqu.head()

CHANNEL             SELLER_NAME  ORIGINAL_INTEREST_RATE  \
LOAN_IDENTIFIER                                                           
100000433226          R   Bank Of America, N.A.                   6.500   
100002773872          C                   Other                   5.375   
100004661943          C  Wells Fargo Bank, N.A.                   5.750   
100005278109          R   Bank Of America, N.A.                   5.625   
100008231057          R          Pnc Bank, N.A.                   5.375   

                 ORIGINAL_UPB  ORIGINAL_LOAN_TERM ORIGINATION_DATE  \
LOAN_IDENTIFIER                                                      
100000433226         153000.0                 360       2008-03-20   
100002773872          75000.0                 360       2008-04-20   
100004661943         166000.0                 360       2008-04-20   
100005278109          33000.0                 180       2008-02-20   
100008231057         288000.0                 240       2008-03-20   

                FIRST_PAYMENT_DATE  ORIGINAL_LTV  ORIGINAL_COMBINED_LTV  \
LOAN_IDENTIFIER                                                           
100000433226            2008-05-20          83.0                   83.0   
100002773872            2008-05-20          53.0                   53.0   
100004661943            2008-06-20          68.0                   88.0   
100005278109            2008-04-20          39.0                   39.0   
100008231057            2008-05-20          80.0                   80.0   

                NUMBER_OF_BORROWERS   DTI  \
LOAN_IDENTIFIER                             
100000433226                      1  35.0   
100002773872                      1  43.0   
100004661943                      2  27.0   
100005278109                      1  28.0   
100008231057                      2  26.0   

                 BORROWER_CREDIT_SCORE_AT_ORIGINATION  \
LOAN_IDENTIFIER                                         
100000433226                                      749   
100002773872                                      705   
100004661943                                      763   
100005278109                                      695   
100008231057                                      758   

                 COBORROWER_CREDIT_SCORE_AT_ORIGINATION  \
LOAN_IDENTIFIER                                           
100000433226                                       <NA>   
100002773872                                       <NA>   
100004661943                                        763   
100005278109                                       <NA>   
100008231057                                        762   

                FIRST_TIME_HOME_BUYER_INDICATOR LOAN_PURPOSE PROPERTY_TYPE  \
LOAN_IDENTIFIER                                                              
100000433226                                  N            C            SF   
100002773872                                  N            P            SF   
100004661943                                  N            R            SF   
100005278109                                  N            R            SF   
100008231057                                  N            R            PU   

                 NUMBER_OF_UNITS OCCUPANCY_STATUS PROPERTY_STATE    MSA  \
LOAN_IDENTIFIER                                                           
100000433226                   1                P             TN  32820   
100002773872                   1                P             PR  41980   
100004661943                   1                P             AZ  38060   
100005278109                   1                P             TX  10180   
100008231057                   1                P             TX  19100   

                ZIP_CODE_SHORT  MORTGAGE_INSURANCE_PERCENTAGE  \
LOAN_IDENTIFIER                                                 
100000433226               381                           12.0   
100002773872               007                            NaN   
1000046619

In [12]:
df_acqu.shape

(444386, 25)

We have ended up with 25 data fields and 444,386 loans.

Now we can pull out the performance data in the same way and sort by both loan identifier and monthly reporting period. We will keep all of the rows this time because each row corresponds to the performance data for a specific month.

In [13]:
df_perf = df_full[perf_cols]

In [14]:
df_perf = df_perf.set_index("LOAN_IDENTIFIER").sort_index()
df_perf = df_perf.sort_values(by=["LOAN_IDENTIFIER", "MONTHLY_REPORTING_PERIOD"])

In [15]:
df_perf.shape

(24313862, 30)

For the performance data, we have over 24 million rows, though these rows correspond to the same 444,386 loans.

In [16]:
df_perf.head()

MONTHLY_REPORTING_PERIOD          SERVICER_NAME  \
LOAN_IDENTIFIER                                                   
100000433226                  2008-04-20  Bank Of America, N.A.   
100000433226                  2008-05-20  Bank Of America, N.A.   
100000433226                  2008-06-20  Bank Of America, N.A.   
100000433226                  2008-07-20  Bank Of America, N.A.   
100000433226                  2008-08-20  Bank Of America, N.A.   

                 CURRENT_INTEREST_RATE  CURRENT_ACTUAL_UPB  LOAN_AGE  \
LOAN_IDENTIFIER                                                        
100000433226                       6.5                 0.0         0   
100000433226                       6.5                 0.0         1   
100000433226                       6.5                 0.0         2   
100000433226                       6.5                 0.0         3   
100000433226                       6.5                 0.0         4   

                 REMAINING_MONTHS_TO_LEGAL_MATURITY  \
LOAN_IDENTIFIER                                       
100000433226                                    360   
100000433226                                    359   
100000433226                                    358   
100000433226                                    357   
100000433226                                    356   

                 REMAINING_MONTHS_TO_MATURITY MATURITY_DATE  \
LOAN_IDENTIFIER                                               
100000433226                              360    2038-04-20   
100000433226                              359    2038-04-20   
100000433226                              358    2038-04-20   
100000433226                              357    2038-04-20   
100000433226                              356    2038-04-20   

                CURRENT_LOAN_DELINQUENCY_STATUS MODIFICATION_FLAG  \
LOAN_IDENTIFIER                                                     
100000433226                                 00                 N   
100000433226                                 00                 N   
100000433226                                 00                 N   
100000433226                                 00                 N   
100000433226                                 00                 N   

                ZERO_BALANCE_CODE ZERO_BALANCE_EFFECTIVE_DATE  \
LOAN_IDENTIFIER                                                 
100000433226                  NaN                         NaT   
100000433226                  NaN                         NaT   
100000433226                  NaN                         NaT   
100000433226                  NaN                         NaT   
100000433226                  NaN                         NaT   

                 UPB_AT_TIME_OF_REMOVAL REPURCHASE_DATE  \
LOAN_IDENTIFIER                                           
100000433226                        NaN             NaT   
100000433226                        NaN             NaT   
100000433226                        NaN             NaT   
100000433226                        NaN             NaT   
100000433226                        NaN             NaT   

                LAST_PAID_INSTALLMENT_DATE FORECLOSURE_DATE DISPOSITION_DATE  \
LOAN_IDENTIFIER                                                                
100000433226                           NaT              NaT              NaT   
100000433226                           NaT              NaT              NaT   
100000433226                           NaT              NaT              NaT   
100000433226                           NaT              NaT              NaT   
100000433226                           NaT              NaT              NaT   

                 FORECLOSURE_COSTS  PROPERTY_PRESERVATION_AND_REPAIR_COSTS  \
LOAN_IDENTIFIER                                                              
100000433226                   NaN                                     NaN   
100000433226                   NaN                              

We can look at a summary of the dataframes.

In [17]:
summarize_dataframe(df_acqu)

Data Type  Missing Values  \
CHANNEL                                         object               0   
SELLER_NAME                                     object               0   
ORIGINAL_INTEREST_RATE                         float32               0   
ORIGINAL_UPB                                   float64               0   
ORIGINAL_LOAN_TERM                               Int16               0   
ORIGINATION_DATE                        datetime64[ns]               0   
FIRST_PAYMENT_DATE                      datetime64[ns]               0   
ORIGINAL_LTV                                   float32               0   
ORIGINAL_COMBINED_LTV                          float32               0   
NUMBER_OF_BORROWERS                           category             175   
DTI                                            float32           13368   
BORROWER_CREDIT_SCORE_AT_ORIGINATION            UInt16             445   
COBORROWER_CREDIT_SCORE_AT_ORIGINATION          UInt16          245446   
FIRST_TIME_HOME_BUYER_INDICATOR               category             166   
LOAN_PURPOSE                                  category               0   
PROPERTY_TYPE                                 category               0   
NUMBER_OF_UNITS                                 UInt16               0   
OCCUPANCY_STATUS                              category               0   
PROPERTY_STATE                                category               0   
MSA                                           category               0   
ZIP_CODE_SHORT                                category               0   
MORTGAGE_INSURANCE_PERCENTAGE                  float32          358489   
AMORTIZATION_TYPE                               object               0   
MORTGAGE_INSURANCE_TYPE                       category          358489   
RELOCATION_MORTGAGE_INDICATOR                 category               0   

                                        Unique Values     count unique  \
CHANNEL                                             3  444386.0      3   
SELLER_NAME                                        16  444386.0     16   
ORIGINAL_INTEREST_RATE                            445  444386.0          
ORIGINAL_UPB                                      727  444386.0          
ORIGINAL_LOAN_TERM                                162  444386.0          
ORIGINATION_DATE                                   72  444386.0          
FIRST_PAYMENT_DATE                                 72  444386.0          
ORIGINAL_LTV                                       97  444386.0          
ORIGINAL_COMBINED_LTV                             113  444386.0          
NUMBER_OF_BORROWERS                                 8  444211.0      8   
DTI                                                64  431018.0          
BORROWER_CREDIT_SCORE_AT_ORIGINATION              371  443941.0          
COBORROWER_CREDIT_SCORE_AT_ORIGINATION            346  198940.0          
FIRST_TIME_HOME_BUYER_INDICATOR                     2  444220.0      2   
LOAN_PURPOSE                                        3  444386.0      3   
PROPERTY_TYPE                                       5  444386.0      5   
NUMBER_OF_UNITS                                     4  444386.0          
OCCUPANCY_STATUS                                    3  444386.0      3   
PROPERTY_STATE                                     54  444386.0     54   
MSA                                               406  444386.0    406   
ZIP_CODE_SHORT                                    898  444386.0    898   
MORTGAGE_INSURANCE_PERCENTAGE                      21   85897.0          
AMORTIZATION_TYPE                                   1  444386.0      1   
MORTGAGE_INSURANCE_TYPE                             2   85897.0      2   
RELOCATION_MORTGAGE_INDICATOR                       2  444386.0      2   

                                                          top    freq  \
CHANNEL                                                     R  183875   
SELLER_NAME                             Bank Of A

In [18]:
summarize_dataframe(df_perf)

Data Type  Missing Values  \
MONTHLY_REPORTING_PERIOD                    datetime64[ns]               0   
SERVICER_NAME                                       object          417742   
CURRENT_INTEREST_RATE                              float32          417694   
CURRENT_ACTUAL_UPB                                 float64               0   
LOAN_AGE                                             Int16          417742   
REMAINING_MONTHS_TO_LEGAL_MATURITY                   Int16          417742   
REMAINING_MONTHS_TO_MATURITY                         Int16         2071055   
MATURITY_DATE                               datetime64[ns]          417742   
CURRENT_LOAN_DELINQUENCY_STATUS                   category               0   
MODIFICATION_FLAG                                 category          417708   
ZERO_BALANCE_CODE                                 category        23896120   
ZERO_BALANCE_EFFECTIVE_DATE                 datetime64[ns]        23896120   
UPB_AT_TIME_OF_REMOVAL                             float32        23896155   
REPURCHASE_DATE                             datetime64[ns]        24313862   
LAST_PAID_INSTALLMENT_DATE                  datetime64[ns]        24290253   
FORECLOSURE_DATE                            datetime64[ns]        24290253   
DISPOSITION_DATE                            datetime64[ns]        24290362   
FORECLOSURE_COSTS                                  float32        24290454   
PROPERTY_PRESERVATION_AND_REPAIR_COSTS             float32        24292833   
ASSET_RECOVERY_COSTS                               float32        24293482   
MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS         float32        24291279   
ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY              float32        24291724   
NET_SALE_PROCEEDS                                  float32        24290613   
CREDIT_ENHANCEMENT_PROCEEDS                        float32        24295127   
REPURCHASE_MAKE_WHOLE_PROCEEDS                     float32        24296156   
OTHER_FORECLOSURE_PROCEEDS                         float32        24294182   
NON_INTEREST_BEARING_UPB                           float32        22659479   
PRINCIPLE_FORGIVENESS_AMOUNT                       float32        24221849   
FORECLOSURE_PRINCIPAL_WRITE_OFF                    float32        24221488   
REPURCHASE_MAKE_WHOLE_PROCEEDS_FLAG               category        23896120   

                                            Unique Values       count unique  \
MONTHLY_REPORTING_PERIOD                              156  24313862.0          
SERVICER_NAME                                          49  23896120.0     49   
CURRENT_INTEREST_RATE                                 535  23896168.0          
CURRENT_ACTUAL_UPB                               12499330  24313862.0          
LOAN_AGE                                              229  23896120.0          
REMAINING_MONTHS_TO_LEGAL_MATURITY                    487  23896120.0          
REMAINING_MONTHS_TO_MATURITY                          363  22242807.0          
MATURITY_DATE                                         546  23896120.0          
CURRENT_LOAN_DELINQUENCY_STATUS                       101  24313862.0    101   
MODIFICATION_FLAG                                       2  23896154.0      2   
ZERO_BALANCE_CODE                                       7    417742.0      7   
ZERO_BALANCE_EFFECTIVE_DATE                           155    417742.0          
UPB_AT_TIME_OF_REMOVAL                             391030    417707.0          
REPURCHASE_DATE                                         0         0.0          
LAST_PAID_INSTALLMENT_DATE                            144     23609.0          
FORECLOSURE_DATE                                      152     23609.0          
DISPOSITION_DATE                                      150     23500.0          
FORECLOSURE_COSTS                                   22412     23408.0          
PROPERTY_PRESERVATION_AND_REPAIR_COSTS              12637     21029.0          
ASSET_RECOVERY_COSTS          

We are going to be "prototyping" code below, and if we try to prototype code with data that is this large, things will take way too long everytime we realize we've messed up and need to start again. So, we instead are going to sample from the data in order to get a smaller dataset. However, we need to be careful because we need to make sure that we look at all of the data for a subset of loans. This means we need to sample from the acquisition data (because each row corresponds to a unique loan), and then use the loan identifiers from our sample to get all of the performance data associated with those loans.

In [21]:
df_acqu = df_acqu.sample(1000, random_state=201)
df_acqu = df_acqu.sort_values(by=["LOAN_IDENTIFIER"])

In [22]:
df_acqu.shape

(1000, 25)

We have sampled 1000 loans. The loans are identified by the `LOAN_IDENTIFIER` column. This is unique per row in the acquisitions data (because you only acquire a loan once), but it repeats for every month of performance data. To pull out performance data, we will use the indices from the subsamples acquisition data. Note that above we set the index of the `df_perf` dataframe to `LOAN_IDENTIFIER` and we sorted the index. This will make subsampling the data much faster. If you have ever used an index in SQL, this is very similar.

In [23]:
df_perf = df_perf.loc[df_acqu.index]

In [24]:
df_acqu

CHANNEL                                        SELLER_NAME  \
LOAN_IDENTIFIER                                                              
100451013296          B                             Suntrust Mortgage Inc.   
101560635874          B                                       Amtrust Bank   
102216046659          C                                 Citimortgage, Inc.   
103546387917          R                                 Citimortgage, Inc.   
106124308559          C  Chase Home Finance Franklin American Mortgage ...   
...                 ...                                                ...   
996914849471          C                              Bank Of America, N.A.   
997317207710          C                                 Citimortgage, Inc.   
998080974186          R                                              Other   
998694206640          C          Jpmorgan Chase Bank, National Association   
998803941475          C                                 Citimortgage, Inc.   

                 ORIGINAL_INTEREST_RATE  ORIGINAL_UPB  ORIGINAL_LOAN_TERM  \
LOAN_IDENTIFIER                                                             
100451013296                      5.750      400000.0                 360   
101560635874                      6.125      153000.0                 360   
102216046659                      6.625       50000.0                 360   
103546387917                      5.750      330000.0                 360   
106124308559                      6.250      122000.0                 360   
...                                 ...           ...                 ...   
996914849471                      5.750      192000.0                 360   
997317207710                      7.250      360000.0                 360   
998080974186                      6.125      317000.0                 360   
998694206640                      5.375      130000.0                 180   
998803941475                      6.500       71000.0                 360   

                ORIGINATION_DATE FIRST_PAYMENT_DATE  ORIGINAL_LTV  \
LOAN_IDENTIFIER                                                     
100451013296          2008-05-20         2008-07-20          72.0   
101560635874          2008-04-20         2008-06-20          62.0   
102216046659          2008-04-20         2008-05-20          28.0   
103546387917          2008-04-20         2008-06-20          73.0   
106124308559          2008-05-20         2008-07-20          80.0   
...                          ...                ...           ...   
996914849471          2008-03-20         2008-05-20          80.0   
997317207710          2008-04-20         2008-06-20          58.0   
998080974186          2008-04-20         2008-06-20          90.0   
998694206640          2008-03-20         2008-05-20          65.0   
998803941475          2008-03-20         2008-05-20          90.0   

                 ORIGINAL_COMBINED_LTV NUMBER_OF_BORROWERS   DTI  \
LOAN_IDENTIFIER                                                    
100451013296                      72.0                   1  42.0   
101560635874                      62.0                   1  59.0   
102216046659                      28.0                   1  23.0   
103546387917                      73.0                   1  60.0   
106124308559                      80.0                   1  23.0   
...                                ...                 ...   ...   
996914849471                      88.0                   1  42.0   
997317207710                      58.0                   1  45.0   
998080974186                      90.0                   2  59.0   
998694206640                      65.0                   2  40.0   
998803941475                      90.0                   1  49.0   

                 BORROWER_CREDIT_SCORE_AT_ORIGINATION  \
LOAN_IDENTIFIER                                         
100451013296                                      709   
101560635874                                 

In [25]:
df_perf

MONTHLY_REPORTING_PERIOD           SERVICER_NAME  \
LOAN_IDENTIFIER                                                    
100451013296                  2008-06-20  Suntrust Mortgage Inc.   
100451013296                  2008-07-20  Suntrust Mortgage Inc.   
100451013296                  2008-08-20  Suntrust Mortgage Inc.   
100451013296                  2008-09-20  Suntrust Mortgage Inc.   
100451013296                  2008-10-20  Suntrust Mortgage Inc.   
...                                  ...                     ...   
998803941475                  2012-05-20      Citimortgage, Inc.   
998803941475                  2012-06-20      Citimortgage, Inc.   
998803941475                  2012-07-20      Citimortgage, Inc.   
998803941475                  2012-08-20      Citimortgage, Inc.   
998803941475                  2012-09-20                     NaN   

                 CURRENT_INTEREST_RATE  CURRENT_ACTUAL_UPB  LOAN_AGE  \
LOAN_IDENTIFIER                                                        
100451013296                      5.75                0.00         0   
100451013296                      5.75                0.00         1   
100451013296                      5.75                0.00         2   
100451013296                      5.75                0.00         3   
100451013296                      5.75                0.00         4   
...                                ...                 ...       ...   
998803941475                      6.50            67504.10        49   
998803941475                      6.50            67420.35        50   
998803941475                      6.50            67336.14        51   
998803941475                      6.50            67251.48        52   
998803941475                       NaN                0.00      <NA>   

                 REMAINING_MONTHS_TO_LEGAL_MATURITY  \
LOAN_IDENTIFIER                                       
100451013296                                    360   
100451013296                                    359   
100451013296                                    358   
100451013296                                    357   
100451013296                                    356   
...                                             ...   
998803941475                                    311   
998803941475                                    310   
998803941475                                    309   
998803941475                                    308   
998803941475                                   <NA>   

                 REMAINING_MONTHS_TO_MATURITY MATURITY_DATE  \
LOAN_IDENTIFIER                                               
100451013296                              359    2038-06-20   
100451013296                              357    2038-06-20   
100451013296                              354    2038-06-20   
100451013296                              356    2038-06-20   
100451013296                              355    2038-06-20   
...                                       ...           ...   
998803941475                              311    2038-04-20   
998803941475                              310    2038-04-20   
998803941475                              309    2038-04-20   
998803941475                              308    2038-04-20   
998803941475                             <NA>           NaT   

                CURRENT_LOAN_DELINQUENCY_STATUS MODIFICATION_FLAG  \
LOAN_IDENTIFIER                                                     
100451013296                                 00                 N   
100451013296                                 00                 N   
100451013296                                 00                 N   
100451013296                                 00                 N   
100451013296                                 00                 N   
...                                         ...               ...   
998803941475                                 00                 N   
998803941475                         

We seem to have the right loans. Let's see how much data we ended up with from our 1000 loans.

In [26]:
df_perf.shape

(54590, 30)

We still have over 50,000 rows in our data set, but this is much more manageable than over 23 million rows (especially for prototyping).

Now that we have our more manageable data set, let's get rid of the full dataset in order to save memory.

In [27]:
del df_full

From here on, we will be following the Fannie Mae provided code (in R) to compute the net loss for each loan. You can find a tutorial [here](https://capitalmarkets.fanniemae.com/media/9066/display) and you can download the R code that we are adapting from [here](https://capitalmarkets.fanniemae.com/media/document/zip/FNMA_SF_Loan_Performance_r_HARP.zip). A number of the steps are heavily reliant on understanding the idiosyncrasies of the data set, so we will not spend a lot of time talking through each step.

We will need the following function to handle missing values in the data.

In [28]:
# Create a function to backfill missing records with the last existing record by setting them to the record prior
def na_lomf(df):
    
    def na_lomf_0(df):
        with np.errstate(invalid='ignore'):
            non_na_idx = np.flatnonzero(~pd.isna(df) & (df > 0))
        if pd.isna(df[0]) or df[0] == 0:
            non_na_idx = np.insert(non_na_idx, 0, 0)
        return np.repeat(df[non_na_idx], np.diff(np.append(non_na_idx, len(df))))
    
    return np.apply_along_axis(na_lomf_0, 0, df)

First, calculate some needed values for the acquisitions data.

In [29]:
df_acqu['CREDIT_SCORE_MIN'] = df_acqu[['BORROWER_CREDIT_SCORE_AT_ORIGINATION', 'COBORROWER_CREDIT_SCORE_AT_ORIGINATION']].min(axis=1)
df_acqu['ORIGINAL_VALUE'] = df_acqu['ORIGINAL_UPB']/(df_acqu['ORIGINAL_LTV']/100.)
df_acqu['ORIGINAL_COMBINED_LTV'].fillna(df_acqu['ORIGINAL_LTV'], inplace=True)

Apply function to backfill missing current actual UPBs and NON_INTEREST_UPB.

In [30]:
df_perf['CURRENT_ACTUAL_UPB'] = df_perf['CURRENT_ACTUAL_UPB'].groupby('LOAN_IDENTIFIER').transform(na_lomf)
df_perf['NON_INTEREST_BEARING_UPB'] = -df_perf['NON_INTEREST_BEARING_UPB'].groupby('LOAN_IDENTIFIER').transform(na_lomf)

Add the original interest rate to the performance data. We can do this because the pandas index is `LOAN_IDENTIFIER` for both dataframes. The join automatically happens on this index. This is known as a one-to-many join, and is common in SQL.

In [31]:
df_perf = df_perf.join(df_acqu['ORIGINAL_INTEREST_RATE'])

The data breaks the principle forgiveness UPB into two categories. For the following exercise, we only need the total. Note that I'm using the `.add` method instead of `+` because `.add` allows for easier handling of missing values. In the below we replace any missing values with `0` before doing the sum.

In [32]:
df_perf['PRINCIPAL_FORGIVENESS_UPB'] = - (df_perf['PRINCIPLE_FORGIVENESS_AMOUNT'].add(df_perf['FORECLOSURE_PRINCIPAL_WRITE_OFF'], fill_value=0))

Calculate the cost of modifying the interest rate, loan terms, and other factors for modified loans.

In [33]:
df_perf['MODIFYING_IR_COST'] = np.where(df_perf['MODIFICATION_FLAG'] == 'Y', 
                                        ((df_perf['ORIGINAL_INTEREST_RATE'].sub(df_perf['CURRENT_INTEREST_RATE'], fill_value=0))/1200.).mul(df_perf['CURRENT_ACTUAL_UPB'], fill_value=0), 
                                        0)
df_perf['MODIFYING_FORGIVEN_INTEREST_COST'] = np.where(df_perf['MODIFICATION_FLAG'] == 'Y', 
                                                      -1 * (df_perf['CURRENT_INTEREST_RATE']/1200).mul(df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0), 
                                                      0)
df_perf['MODIFYING_FORGIVEN_COST'] = -df_perf.groupby('LOAN_IDENTIFIER')['PRINCIPAL_FORGIVENESS_UPB'].min()

df_perf['CUM_MODIFYING_IR_COST'] = df_perf.groupby('LOAN_IDENTIFIER')['MODIFYING_IR_COST'].sum()
df_perf['CUM_MODIFYING_FORGIVEN_INTEREST_COST'] = df_perf.groupby('LOAN_IDENTIFIER')['MODIFYING_FORGIVEN_INTEREST_COST'].sum()

At this point, we no longer need the loan performance data for every month. The modification costs summarize the data we need. So, we will now just keep the last row for each loan.

In [34]:
df_perf = df_perf.groupby('LOAN_IDENTIFIER').tail(1)

In [35]:
df_perf.shape

(1000, 37)

You can see that our performance data now has exactly as many rows as our acquisitions data.

We no longer need a few columns, so we will get rid of them.

In [36]:
df_perf.drop(['PRINCIPLE_FORGIVENESS_AMOUNT', 'FORECLOSURE_PRINCIPAL_WRITE_OFF'], axis=1, inplace=True)

We will now primarily be looking at delinquency codes. We will standardize them in order to work with them more easily.

In [37]:
df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] = df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'].replace("XX", "999")

We will also get the final unpaid balance for each loan.

In [38]:
df_perf['FINAL_UPB'] = df_perf['CURRENT_ACTUAL_UPB'].add(df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0).add(df_perf['PRINCIPAL_FORGIVENESS_UPB'], fill_value=0)

We will find it useful to compute a `LAST_STAT` variable that summarizes the last state of the loan. Each loan takes care of more and more specialized cases.

In [39]:
df_perf['LAST_STAT'] = df_perf['ZERO_BALANCE_CODE'].replace({'01': 'P', '02': 'T', '03': 'S', '06': 'R', '09': 'F', '15': 'N', '16': 'L'})
df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] == '999'), ['LAST_STAT']] = 'X'
df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'].astype(float) > 9), ['LAST_STAT']] = '9'
df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] == '0'), ['LAST_STAT']] = 'C'
df_perf.loc[pd.isna(df_perf['LAST_STAT']), ['LAST_STAT']] = df_perf.loc[pd.isna(df_perf['LAST_STAT']), ['CURRENT_LOAN_DELINQUENCY_STATUS']].values

We will need to find out the difference between two months in terms of number of months (i.e. difference between current data and when the last payment was made). This helper function will help with that.

In [40]:
def diff_month(dates):
    if pd.isna(dates[0]) or pd.isna(dates[1]):
        return 0
    return (dates[0].year - dates[1].year) * 12 + dates[0].month - dates[1].month

We will use the above to calculate some intermediate variables.

In [41]:
df_perf['lpi2disp'] = df_perf[['DISPOSITION_DATE', 'LAST_PAID_INSTALLMENT_DATE']].apply(diff_month, axis=1)
df_perf['zb2disp'] = df_perf[['DISPOSITION_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE']].apply(diff_month, axis=1)

Now, we need to define which of our `LAST_STAT` is a credit event, which will be associated with losses.

In [42]:
credit_events = ['F', 'S', 'T', 'N']

Finally, we can compute a number of loss related variables for the loans that have a credit event.

In [43]:
df_perf['INT_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                              np.fmax(df_perf['FINAL_UPB'].mul(((df_perf['CURRENT_INTEREST_RATE']/100) - .0035)/12, fill_value=0).mul(df_perf['lpi2disp'], fill_value=0), 0),
                              0)

df_perf['total_expense'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                              df_perf[['FORECLOSURE_COSTS', 'PROPERTY_PRESERVATION_AND_REPAIR_COSTS', 'ASSET_RECOVERY_COSTS', 'ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY', 'MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS']].sum(axis=1),
                              0)
df_perf['total_proceeds'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                              -1*df_perf[['NET_SALE_PROCEEDS', 'CREDIT_ENHANCEMENT_PROCEEDS', 'REPURCHASE_MAKE_WHOLE_PROCEEDS', 'OTHER_FORECLOSURE_PROCEEDS']].sum(axis=1),
                              0)

df_perf['NET_LOSS'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                              df_perf[['CURRENT_ACTUAL_UPB', 'INT_COST', 'total_expense', 'total_proceeds']].sum(axis=1),
                              0)
df_perf['NET_SEVERITY'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                              df_perf[['CURRENT_ACTUAL_UPB', 'INT_COST', 'total_expense', 'total_proceeds']].sum(axis=1)/df_perf['CURRENT_ACTUAL_UPB'],
                              0) 

df_perf['MODIR_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']) & (df_perf['MODIFICATION_FLAG'] == 'Y'),
                                df_perf['zb2disp'].mul((df_perf['ORIGINAL_INTEREST_RATE'].sub(df_perf['CURRENT_INTEREST_RATE'], fill_value=0))/1200., fill_value=0).mul(df_perf['CURRENT_ACTUAL_UPB'], fill_value=0),
                                0) + df_perf['CUM_MODIFYING_IR_COST']
df_perf['MODFB_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']) & (df_perf['MODIFICATION_FLAG'] == 'Y'),
                                df_perf['zb2disp'].mul(df_perf['CURRENT_INTEREST_RATE']/1200., fill_value=0).mul(-1*df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0),
                                0) + df_perf['CUM_MODIFYING_FORGIVEN_INTEREST_COST']
df_perf['LOAN_MODIFICATION_COSTS'] = df_perf[['MODIFYING_FORGIVEN_COST', 'MODIR_COST', 'MODFB_COST']].sum(axis=1)
df_perf['TOTAL_LOSSES'] = df_perf[['NET_LOSS', 'LOAN_MODIFICATION_COSTS']].sum(axis=1)

As a last step, we want to join our loss related variables with our acquisitions data so that we can use acquisition data (which is generally all we'll have on a new loan) to predict net losses. We also include `MSA` identifiers which are geographic identification codes for the loans.

In [44]:
df_small = df_acqu.join(df_perf[['ZERO_BALANCE_CODE', 'LOAN_AGE', 'NET_LOSS', 'NET_SEVERITY', 'LAST_STAT', 'LOAN_MODIFICATION_COSTS', 'TOTAL_LOSSES']])

Now let's summarize.

In [45]:
summarize_dataframe(df_small)

Data Type  Missing Values  \
CHANNEL                                         object               0   
SELLER_NAME                                     object               0   
ORIGINAL_INTEREST_RATE                         float32               0   
ORIGINAL_UPB                                   float64               0   
ORIGINAL_LOAN_TERM                               Int16               0   
ORIGINATION_DATE                        datetime64[ns]               0   
FIRST_PAYMENT_DATE                      datetime64[ns]               0   
ORIGINAL_LTV                                   float32               0   
ORIGINAL_COMBINED_LTV                          float32               0   
NUMBER_OF_BORROWERS                           category               0   
DTI                                            float32              28   
BORROWER_CREDIT_SCORE_AT_ORIGINATION            UInt16               0   
COBORROWER_CREDIT_SCORE_AT_ORIGINATION          UInt16             563   
FIRST_TIME_HOME_BUYER_INDICATOR               category               0   
LOAN_PURPOSE                                  category               0   
PROPERTY_TYPE                                 category               0   
NUMBER_OF_UNITS                                 UInt16               0   
OCCUPANCY_STATUS                              category               0   
PROPERTY_STATE                                category               0   
MSA                                           category               0   
ZIP_CODE_SHORT                                category               0   
MORTGAGE_INSURANCE_PERCENTAGE                  float32             811   
AMORTIZATION_TYPE                               object               0   
MORTGAGE_INSURANCE_TYPE                       category             811   
RELOCATION_MORTGAGE_INDICATOR                 category               0   
CREDIT_SCORE_MIN                               float64               0   
ORIGINAL_VALUE                                 float64               0   
ZERO_BALANCE_CODE                             category              70   
LOAN_AGE                                         Int16             930   
NET_LOSS                                       float64               0   
NET_SEVERITY                                   float64               0   
LAST_STAT                                       object               0   
LOAN_MODIFICATION_COSTS                        float64               0   
TOTAL_LOSSES                                   float64               0   

                                        Unique Values   count unique    top  \
CHANNEL                                             3  1000.0      3      R   
SELLER_NAME                                        16  1000.0     16  Other   
ORIGINAL_INTEREST_RATE                             38  1000.0                 
ORIGINAL_UPB                                      332  1000.0                 
ORIGINAL_LOAN_TERM                                  9  1000.0                 
ORIGINATION_DATE                                   12  1000.0                 
FIRST_PAYMENT_DATE                                 11  1000.0                 
ORIGINAL_LTV                                       82  1000.0                 
ORIGINAL_COMBINED_LTV                              82  1000.0                 
NUMBER_OF_BORROWERS                                 4  1000.0      4      2   
DTI                                                60   972.0                 
BORROWER_CREDIT_SCORE_AT_ORIGINATION              206  1000.0                 
COBORROWER_CREDIT_SCORE_AT_ORIGINATION            166   437.0                 
FIRST_TIME_HOME_BUYER_INDICATOR                     2  1000.0      2      N   
LOAN_PURPOSE                                        3  1000.0      3      C   
PROPERTY_TYPE                                       5  1000.0      5     SF   
NUMBER_OF_UNITS                                     4  1000.0                 
OCCUPANCY_STATUS                

## Putting It All Together

The above code is fine, but it is unwieldy if you want to do this with a bunch of datasets (for example all quarters for all years). So, it's often a good idea to take a bunch of code that you have prototyped and put it into a function that encapsulates everything. That way you can run the whole thing in one go, and you can apply it to a bunch of different files easily. Below we do that. You specify the filename, and it does everything we do above in one go. However, we don't sample from the data set, so it will do the merge on the whole data set.

In [46]:
def modify_data(file_name):

    if not os.path.isfile(file_name):
        return

    df_full = pd.read_csv(file_name,
                          header=None,
                          names=col_names,
                          dtype=col_classes,
                          parse_dates=date_columns,
                          delimiter='|')

    # df_full = df_full.set_index('LOAN_IDENTIFIER').sort_index()
    # df_full = df_full.sort_values(by=['LOAN_IDENTIFIER', 'MONTHLY_REPORTING_PERIOD'])

    df_acqu = df_full[acquisition_columns]

    df_acqu = df_acqu.sort_values(by=["LOAN_IDENTIFIER", "MONTHLY_REPORTING_PERIOD"]).groupby('LOAN_IDENTIFIER').head(1)
    df_acqu = df_acqu.set_index('LOAN_IDENTIFIER').sort_index()
    df_acqu.drop(["MONTHLY_REPORTING_PERIOD"], axis=1, inplace=True)

    # Not Sure I want to do this, but they do it.
    # Delete unnecessary Acquisition variables.
    #df_acqu.drop(['SELLER_NAME', 'PRODUCT_TYPE'], axis=1, inplace=True)

    # Obtain the Minimum Fico Score of the Borrower and Co-Borrower, Calculate House Price, and Replace Missing OCLTV values with OLTV values where available
    df_acqu['CREDIT_SCORE_MIN'] = df_acqu[['BORROWER_CREDIT_SCORE_AT_ORIGINATION', 'COBORROWER_CREDIT_SCORE_AT_ORIGINATION']].min(axis=1)
    df_acqu['ORIGINAL_VALUE'] = df_acqu['ORIGINAL_UPB']/(df_acqu['ORIGINAL_LTV']/100.)
    df_acqu['ORIGINAL_COMBINED_LTV'].fillna(df_acqu['ORIGINAL_LTV'], inplace=True)

    df_perf = df_full[perf_cols]

    del df_full

    df_perf = df_perf.set_index('LOAN_IDENTIFIER').sort_index()
    df_perf = df_perf.sort_values(by=['LOAN_IDENTIFIER', 'MONTHLY_REPORTING_PERIOD'])

    # Apply function to backfill missing current actual UPBs and NON_INTEREST_UPB
    df_perf['CURRENT_ACTUAL_UPB'] = df_perf['CURRENT_ACTUAL_UPB'].groupby('LOAN_IDENTIFIER').transform(na_lomf)
    df_perf['NON_INTEREST_BEARING_UPB'] = -df_perf['NON_INTEREST_BEARING_UPB'].groupby('LOAN_IDENTIFIER').transform(na_lomf)

    df_perf = df_perf.join(df_acqu['ORIGINAL_INTEREST_RATE'])

    # LLPUB 16.2 release breaks the principle forgiveness UPB into two categories. 
    # For the following exercise, only need the total
    df_perf['PRINCIPAL_FORGIVENESS_UPB'] = - (df_perf['PRINCIPLE_FORGIVENESS_AMOUNT'].add(df_perf['FORECLOSURE_PRINCIPAL_WRITE_OFF'], fill_value=0))

    # Calculate the cost of modifying the interest rate
    df_perf['MODIFYING_IR_COST'] = np.where(df_perf['MODIFICATION_FLAG'] == 'Y', 
                                            ((df_perf['ORIGINAL_INTEREST_RATE'].sub(df_perf['CURRENT_INTEREST_RATE'], fill_value=0))/1200.).mul(df_perf['CURRENT_ACTUAL_UPB'], fill_value=0), 
                                            0)
    df_perf['MODIFYING_FORGIVEN_INTEREST_COST'] = np.where(df_perf['MODIFICATION_FLAG'] == 'Y', 
                                                          -1 * (df_perf['CURRENT_INTEREST_RATE']/1200).mul(df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0), 
                                                          0)
    df_perf['MODIFYING_FORGIVEN_COST'] = -df_perf.groupby('LOAN_IDENTIFIER')['PRINCIPAL_FORGIVENESS_UPB'].min()

    df_perf['CUM_MODIFYING_IR_COST'] = df_perf.groupby('LOAN_IDENTIFIER')['MODIFYING_IR_COST'].sum()
    df_perf['CUM_MODIFYING_FORGIVEN_INTEREST_COST'] = df_perf.groupby('LOAN_IDENTIFIER')['MODIFYING_FORGIVEN_INTEREST_COST'].sum()

    df_perf = df_perf.groupby('LOAN_IDENTIFIER').tail(1)


    df_perf.drop(['PRINCIPLE_FORGIVENESS_AMOUNT', 'FORECLOSURE_PRINCIPAL_WRITE_OFF'], axis=1, inplace=True)

    # Standardize Delinquency Status Codes
    df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] = df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'].replace("XX", "999")





    # df_perf['MODIFIED_TERM_FLAG'] = df.groupby('LOAN_IDENTIFIER').apply(lambda df: ((df['MATURITY_DATE'].nunique() > 1) & (df['MODIFICATION_FLAG'] == "Y")).max())
    # They never use the below, so I'm not going to.
    # df_perf['MODIFIED_UPB_FLAG'] = df.groupby('LOAN_IDENTIFIER').apply(lambda df: )

    df_perf['FINAL_UPB'] = df_perf['CURRENT_ACTUAL_UPB'].add(df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0).add(df_perf['PRINCIPAL_FORGIVENESS_UPB'], fill_value=0)


    df_perf['LAST_STAT'] = df_perf['ZERO_BALANCE_CODE'].replace({'01': 'P', '02': 'T', '03': 'S', '06': 'R', '09': 'F', '15': 'N', '16': 'L'})
    df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] == '999'), ['LAST_STAT']] = 'X'
    df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'].astype(float) > 9), ['LAST_STAT']] = '9'
    df_perf.loc[(pd.isna(df_perf['LAST_STAT'])) & (df_perf['CURRENT_LOAN_DELINQUENCY_STATUS'] == '0'), ['LAST_STAT']] = 'C'
    df_perf.loc[pd.isna(df_perf['LAST_STAT']), ['LAST_STAT']] = df_perf.loc[pd.isna(df_perf['LAST_STAT']), ['CURRENT_LOAN_DELINQUENCY_STATUS']].values

    def diff_month(dates):
        if pd.isna(dates[0]) or pd.isna(dates[1]):
            return 0
        return (dates[0].year - dates[1].year) * 12 + dates[0].month - dates[1].month

    df_perf['lpi2disp'] = df_perf[['DISPOSITION_DATE', 'LAST_PAID_INSTALLMENT_DATE']].apply(diff_month, axis=1)
    df_perf['zb2disp'] = df_perf[['DISPOSITION_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE']].apply(diff_month, axis=1)

    credit_events = ['F', 'S', 'T', 'N']

    df_perf['INT_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                                  np.fmax(df_perf['FINAL_UPB'].mul(((df_perf['CURRENT_INTEREST_RATE']/100) - .0035)/12, fill_value=0).mul(df_perf['lpi2disp'], fill_value=0), 0),
                                  0)
    
    df_perf['total_expense'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                                  df_perf[['FORECLOSURE_COSTS', 'PROPERTY_PRESERVATION_AND_REPAIR_COSTS', 'ASSET_RECOVERY_COSTS', 'ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY', 'MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS']].sum(axis=1),
                                  0)
    df_perf['total_proceeds'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                                  -1*df_perf[['NET_SALE_PROCEEDS', 'CREDIT_ENHANCEMENT_PROCEEDS', 'REPURCHASE_MAKE_WHOLE_PROCEEDS', 'OTHER_FORECLOSURE_PROCEEDS']].sum(axis=1),
                                  0)
    
    df_perf['NET_LOSS'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                                  df_perf[['CURRENT_ACTUAL_UPB', 'INT_COST', 'total_expense', 'total_proceeds']].sum(axis=1),
                                  0)
    df_perf['NET_SEVERITY'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']), 
                                  df_perf[['CURRENT_ACTUAL_UPB', 'INT_COST', 'total_expense', 'total_proceeds']].sum(axis=1)/df_perf['CURRENT_ACTUAL_UPB'],
                                  0) 
    
    df_perf['MODIR_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']) & (df_perf['MODIFICATION_FLAG'] == 'Y'),
                                    df_perf['zb2disp'].mul((df_perf['ORIGINAL_INTEREST_RATE'].sub(df_perf['CURRENT_INTEREST_RATE'], fill_value=0))/1200., fill_value=0).mul(df_perf['CURRENT_ACTUAL_UPB'], fill_value=0),
                                    0) + df_perf['CUM_MODIFYING_IR_COST']
    df_perf['MODFB_COST'] = np.where(df_perf['LAST_STAT'].isin(credit_events) & ~pd.isna(df_perf['DISPOSITION_DATE']) & (df_perf['MODIFICATION_FLAG'] == 'Y'),
                                    df_perf['zb2disp'].mul(df_perf['CURRENT_INTEREST_RATE']/1200., fill_value=0).mul(-1*df_perf['NON_INTEREST_BEARING_UPB'], fill_value=0),
                                    0) + df_perf['CUM_MODIFYING_FORGIVEN_INTEREST_COST']
    df_perf['LOAN_MODIFICATION_COSTS'] = df_perf[['MODIFYING_FORGIVEN_COST', 'MODIR_COST', 'MODFB_COST']].sum(axis=1)
    df_perf['TOTAL_LOSSES'] = df_perf[['NET_LOSS', 'LOAN_MODIFICATION_COSTS']].sum(axis=1)

    # df_perf = df_perf[['MSA', 'ZERO_BALANCE_CODE', 'LOAN_AGE', 'NET_LOSS', 'NET_SEVERITY', 'LAST_STAT']]

    return df_acqu.join(df_perf[['ZERO_BALANCE_CODE', 'LOAN_AGE', 'NET_LOSS', 'NET_SEVERITY', 'LAST_STAT', 'LOAN_MODIFICATION_COSTS', 'TOTAL_LOSSES']])

Let's try it on our data. This will take some time.

In [47]:
%%time
df = modify_data(FILES_LOCATION + "2008Q2.csv")

CPU times: user 8min 50s, sys: 34 s, total: 9min 24s
Wall time: 9min 24s


Let's go ahead and summarize this dataframe.

In [48]:
summarize_dataframe(df)

Data Type  Missing Values  \
CHANNEL                                         object               0   
SELLER_NAME                                     object               0   
ORIGINAL_INTEREST_RATE                         float32               0   
ORIGINAL_UPB                                   float64               0   
ORIGINAL_LOAN_TERM                               Int16               0   
ORIGINATION_DATE                        datetime64[ns]               0   
FIRST_PAYMENT_DATE                      datetime64[ns]               0   
ORIGINAL_LTV                                   float32               0   
ORIGINAL_COMBINED_LTV                          float32               0   
NUMBER_OF_BORROWERS                           category             175   
DTI                                            float32           13368   
BORROWER_CREDIT_SCORE_AT_ORIGINATION            UInt16             445   
COBORROWER_CREDIT_SCORE_AT_ORIGINATION          UInt16          245446   
FIRST_TIME_HOME_BUYER_INDICATOR               category             166   
LOAN_PURPOSE                                  category               0   
PROPERTY_TYPE                                 category               0   
NUMBER_OF_UNITS                                 UInt16               0   
OCCUPANCY_STATUS                              category               0   
PROPERTY_STATE                                category               0   
MSA                                           category               0   
ZIP_CODE_SHORT                                category               0   
MORTGAGE_INSURANCE_PERCENTAGE                  float32          358489   
AMORTIZATION_TYPE                               object               0   
MORTGAGE_INSURANCE_TYPE                       category          358489   
RELOCATION_MORTGAGE_INDICATOR                 category               0   
CREDIT_SCORE_MIN                               float64             328   
ORIGINAL_VALUE                                 float64               0   
ZERO_BALANCE_CODE                             category           26644   
LOAN_AGE                                         Int16          417742   
NET_LOSS                                       float64               0   
NET_SEVERITY                                   float64               0   
LAST_STAT                                       object               0   
LOAN_MODIFICATION_COSTS                        float64               0   
TOTAL_LOSSES                                   float64               0   

                                        Unique Values     count unique  \
CHANNEL                                             3  444386.0      3   
SELLER_NAME                                        16  444386.0     16   
ORIGINAL_INTEREST_RATE                            445  444386.0          
ORIGINAL_UPB                                      727  444386.0          
ORIGINAL_LOAN_TERM                                162  444386.0          
ORIGINATION_DATE                                   72  444386.0          
FIRST_PAYMENT_DATE                                 72  444386.0          
ORIGINAL_LTV                                       97  444386.0          
ORIGINAL_COMBINED_LTV                             113  444386.0          
NUMBER_OF_BORROWERS                                 8  444211.0      8   
DTI                                                64  431018.0          
BORROWER_CREDIT_SCORE_AT_ORIGINATION              371  443941.0          
COBORROWER_CREDIT_SCORE_AT_ORIGINATION            346  198940.0          
FIRST_TIME_HOME_BUYER_INDICATOR                     2  444220.0      2   
LOAN_PURPOSE                                        3  444386.0      3   
PROPERTY_TYPE                                       5  444386.0      5   
NUMBER_OF_UNITS                                     4  444386.0          
OCCUPANCY_STATUS                                    3  444386.0      3   
PROPERTY_STATE                                  

## Adding MSA Data

We will also find it helpful to have more specific data about the MSA each loan originated in. So, we will take our final dataframe `df`, and merge it with some MSA specific data.

In [49]:
df_msa = pd.read_csv('MSA_name_pop_data.csv')

In [50]:
df_msa.head()

MSA                     MSA_Name  CENSUS_2010_POP
0  10180                  Abilene, TX           165252
1  10420                    Akron, OH           703200
2  10500                   Albany, GA           157308
3  10540                   Albany, OR           116672
4  10580  Albany-Schenectady-Troy, NY           870716

In [51]:
summarize_dataframe(df_msa)

Data Type  Missing Values  Unique Values  count unique  \
MSA                 int64               0            381  381.0          
MSA_Name           object               0            381  381.0    381   
CENSUS_2010_POP     int64               0            381  381.0          

                         top freq           mean             std      min  \
MSA                                 29694.566929    11387.924774  10180.0   
MSA_Name         Abilene, TX    1                                           
CENSUS_2010_POP                    688850.740157  1575670.666621  55274.0   

                      25%       50%       75%         max  
MSA               19740.0   29420.0   39740.0     49740.0  
MSA_Name                                                   
CENSUS_2010_POP  138333.0  230162.0  549475.0  19567410.0

In order to join it with our other dataframe, we need to set `MSA` as the index. Then sorting by the index will speed up the join.

In [52]:
df_msa = df_msa.set_index('MSA').sort_index()

Right now, the data type for `MSA` in `df` is a "category". This was useful when we were working with the performance data above, but in order to do the join the data types for the joining variable must be the same (it is "int64" for `df_msa`). Below we convert `MSA` into a "int64" type datatype.

In [53]:
df['MSA'] = df['MSA'].astype('int64')

Now we can do the join. We need to specify that we are joining on 'MSA'. The join function automatically uses the index of the dataframe inside the `.join` method (this is `df_msa` below), but you can choose what column on the calling dataframe (`df` below) you join on. So, below we are choosing to join on 'MSA' for the calling dataframe, and it is using the index we set to be `MSA` for `df_msa`.

In [54]:
df = df.join(df_msa, on='MSA')

We can see we have two new columns, `MSA_Name` and `CENSUS_2010_POP`.

In [55]:
summarize_dataframe(df)

Data Type  Missing Values  \
CHANNEL                                         object               0   
SELLER_NAME                                     object               0   
ORIGINAL_INTEREST_RATE                         float32               0   
ORIGINAL_UPB                                   float64               0   
ORIGINAL_LOAN_TERM                               Int16               0   
ORIGINATION_DATE                        datetime64[ns]               0   
FIRST_PAYMENT_DATE                      datetime64[ns]               0   
ORIGINAL_LTV                                   float32               0   
ORIGINAL_COMBINED_LTV                          float32               0   
NUMBER_OF_BORROWERS                           category             175   
DTI                                            float32           13368   
BORROWER_CREDIT_SCORE_AT_ORIGINATION            UInt16             445   
COBORROWER_CREDIT_SCORE_AT_ORIGINATION          UInt16          245446   
FIRST_TIME_HOME_BUYER_INDICATOR               category             166   
LOAN_PURPOSE                                  category               0   
PROPERTY_TYPE                                 category               0   
NUMBER_OF_UNITS                                 UInt16               0   
OCCUPANCY_STATUS                              category               0   
PROPERTY_STATE                                category               0   
MSA                                              int64               0   
ZIP_CODE_SHORT                                category               0   
MORTGAGE_INSURANCE_PERCENTAGE                  float32          358489   
AMORTIZATION_TYPE                               object               0   
MORTGAGE_INSURANCE_TYPE                       category          358489   
RELOCATION_MORTGAGE_INDICATOR                 category               0   
CREDIT_SCORE_MIN                               float64             328   
ORIGINAL_VALUE                                 float64               0   
ZERO_BALANCE_CODE                             category           26644   
LOAN_AGE                                         Int16          417742   
NET_LOSS                                       float64               0   
NET_SEVERITY                                   float64               0   
LAST_STAT                                       object               0   
LOAN_MODIFICATION_COSTS                        float64               0   
TOTAL_LOSSES                                   float64               0   
MSA_Name                                        object           70834   
CENSUS_2010_POP                                float64           70834   

                                        Unique Values     count unique  \
CHANNEL                                             3  444386.0      3   
SELLER_NAME                                        16  444386.0     16   
ORIGINAL_INTEREST_RATE                            445  444386.0          
ORIGINAL_UPB                                      727  444386.0          
ORIGINAL_LOAN_TERM                                162  444386.0          
ORIGINATION_DATE                                   72  444386.0          
FIRST_PAYMENT_DATE                                 72  444386.0          
ORIGINAL_LTV                                       97  444386.0          
ORIGINAL_COMBINED_LTV                             113  444386.0          
NUMBER_OF_BORROWERS                                 8  444211.0      8   
DTI                                                64  431018.0          
BORROWER_CREDIT_SCORE_AT_ORIGINATION              371  443941.0          
COBORROWER_CREDIT_SCORE_AT_ORIGINATION            346  198940.0          
FIRST_TIME_HOME_BUYER_INDICATOR                     2  444220.0      2   
LOAN_PURPOSE                                        3  444386.0      3   
PROPERTY_TYPE                                       5  444386.0      5   
NUMBER_OF_UNITS                                 

We will rename the column `MSA_Name` to `MSA_NAME` to be consistent with our all capital letter column naming scheme. Note that this is not essential, but it is good to practice to maintain consistency in naming schemes.

In [56]:
df.rename(columns={"MSA_Name": "MSA_NAME"}, inplace=True)

In [57]:
df.head()

CHANNEL             SELLER_NAME  ORIGINAL_INTEREST_RATE  \
LOAN_IDENTIFIER                                                           
100000433226          R   Bank Of America, N.A.                   6.500   
100002773872          C                   Other                   5.375   
100004661943          C  Wells Fargo Bank, N.A.                   5.750   
100005278109          R   Bank Of America, N.A.                   5.625   
100008231057          R          Pnc Bank, N.A.                   5.375   

                 ORIGINAL_UPB  ORIGINAL_LOAN_TERM ORIGINATION_DATE  \
LOAN_IDENTIFIER                                                      
100000433226         153000.0                 360       2008-03-20   
100002773872          75000.0                 360       2008-04-20   
100004661943         166000.0                 360       2008-04-20   
100005278109          33000.0                 180       2008-02-20   
100008231057         288000.0                 240       2008-03-20   

                FIRST_PAYMENT_DATE  ORIGINAL_LTV  ORIGINAL_COMBINED_LTV  \
LOAN_IDENTIFIER                                                           
100000433226            2008-05-20          83.0                   83.0   
100002773872            2008-05-20          53.0                   53.0   
100004661943            2008-06-20          68.0                   88.0   
100005278109            2008-04-20          39.0                   39.0   
100008231057            2008-05-20          80.0                   80.0   

                NUMBER_OF_BORROWERS   DTI  \
LOAN_IDENTIFIER                             
100000433226                      1  35.0   
100002773872                      1  43.0   
100004661943                      2  27.0   
100005278109                      1  28.0   
100008231057                      2  26.0   

                 BORROWER_CREDIT_SCORE_AT_ORIGINATION  \
LOAN_IDENTIFIER                                         
100000433226                                      749   
100002773872                                      705   
100004661943                                      763   
100005278109                                      695   
100008231057                                      758   

                 COBORROWER_CREDIT_SCORE_AT_ORIGINATION  \
LOAN_IDENTIFIER                                           
100000433226                                       <NA>   
100002773872                                       <NA>   
100004661943                                        763   
100005278109                                       <NA>   
100008231057                                        762   

                FIRST_TIME_HOME_BUYER_INDICATOR LOAN_PURPOSE PROPERTY_TYPE  \
LOAN_IDENTIFIER                                                              
100000433226                                  N            C            SF   
100002773872                                  N            P            SF   
100004661943                                  N            R            SF   
100005278109                                  N            R            SF   
100008231057                                  N            R            PU   

                 NUMBER_OF_UNITS OCCUPANCY_STATUS PROPERTY_STATE    MSA  \
LOAN_IDENTIFIER                                                           
100000433226                   1                P             TN  32820   
100002773872                   1                P             PR  41980   
100004661943                   1                P             AZ  38060   
100005278109                   1                P             TX  10180   
100008231057                   1                P             TX  19100   

                ZIP_CODE_SHORT  MORTGAGE_INSURANCE_PERCENTAGE  \
LOAN_IDENTIFIER                                                 
100000433226               381                           12.0   
100002773872               007                            NaN   
1000046619

Now our dataframe is complete! We can then write out the new dataset as a CSV file. We will want to use a data delimiter besides commas because a number of our data fields (including `SELLER_NAME` and `MSA_NAME`) have commas in the data. So we will use the seperator that out dataset originally had `|`.

In [ ]:
df.to_csv('./processed_dataset.csv', sep='|')